# Imports

In [42]:
##############
# basic libs #
##############

from subprocess import call
from tqdm import *
from time import time
import warnings
warnings.filterwarnings('ignore')
import os, sys, time, datetime, json, string, glob, re

############
# plumbing #
############

import tempfile
from joblib import Memory
cachedir = tempfile.mkdtemp()
mem = Memory(cachedir=cachedir, verbose=1)

from sqlalchemy import create_engine
from collections import OrderedDict
import urllib2, requests, operator, hashlib, uuid, itertools

###########
# science #
###########

import scipy as sp
import numpy as np
import pandas as pd

#######
# ML #
######

import pymc3 as pm
import theano as thno
import theano.tensor as T
from sklearn import preprocessing
from sklearn import metrics
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import pipeline
from sklearn import feature_selection

#################
# visualization #
#################

# plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=False, world_readable=True, theme='pearl')
tls.set_credentials_file(username=os.environ.get('PLOTLY_USERNAME'), api_key=os.environ.get('PLOTLY_APIKEY'))

# matplotlib
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# seaborn
import seaborn as sns
sns.set(style="dark", palette="muted")
sns.set_context("notebook",
                font_scale=1.5,
                rc={"lines.linewidth": 2.5})

#graphistry
import graphistry
graphistry.register(key=os.environ.get('GRAPHISTRY_APIKEY'))

In [43]:
%reload_ext watermark

In [44]:
%watermark -a "Ken Cavagnolo" -n -u -v -m -h -g -p numpy,scipy,pandas,seaborn,scikit-learn

Ken Cavagnolo 
Last updated: Tue Feb 16 2016 

CPython 2.7.10
IPython 4.0.3

numpy 1.10.4
scipy 0.17.0
pandas 0.17.1
seaborn 0.7.0
scikit-learn 0.17

compiler   : GCC 5.2.1 20151010
system     : Linux
release    : 4.2.0-23-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
host name  : ubuntu
Git hash   : a2c2b27acf49b258e1702aa8ce17d34fbb0bd4d0


In [45]:
def kscore(y_pred, y_true):
    nrows = y_pred.shape[0]
    y_pred = y_pred + 1e-15
    probs = y_pred[list(range(nrows)), y_true.astype(int)]
    return - np.sum(np.log(probs)) / nrows

In [46]:
def factors(n):    
    return set(reduce(list.__add__,
                      ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

In [47]:
def feature_indices(df, attr):
    idx = {}
    for a in attr:
        idx[a] = [c for c in df.columns if c.startswith(a)]
    return idx

In [48]:
def all_combos(attr):
    combos = []
    for i in range(1, len(attr)+1):
        for subset in itertools.combinations(attr, i):
            combos.append(subset)
    return combos

In [98]:
@mem.cache
def run_cv(df, attr, clf_class, **kwargs):
    
    # scoring arrays
    acc = []
    prec = []
    recl = []
    f1 = []
    
    # get attribute combos to explore
    attr_com = all_combos(attr)

    # get these attr as indices
    attr_idx = feature_indices(df, attr)
    
    # iterate through all combinations of features
    for a in attr_com:
        idxs = []
        for b in a:
            idxs.extend(attr_idx[b])
        X = df.ix[:, (idxs)].copy().values
        y = df.fault_severity.values
    
        # run kfold cv
        kf = cross_validation.KFold(len(y), n_folds=10, shuffle=True)
        y_pred = y.copy()
        mean_acc = 0.0
        mean_prec = 0.0
        mean_recl = 0.0
        mean_f1 = 0.0
        for train_index, test_index in kf:
            clf = clf_class(**kwargs)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)
            y_pred[test_index] = clf.predict(X_test)
            mean_acc += metrics.accuracy_score(y[test_index], y_pred[test_index])
            mean_recl += metrics.recall_score(y[test_index], y_pred[test_index])
            mean_prec += metrics.precision_score(y[test_index], y_pred[test_index])
            mean_f1 += metrics.f1_score(y[test_index], y_pred[test_index])
        acc.append(mean_acc / len(kf))
        recl.append(mean_recl / len(kf))
        prec.append(mean_prec / len(kf))
        f1.append(mean_f1 / len(kf))
    result = pd.DataFrame({'attr_combo':attr_com, 'accuracy':acc, 'precision':prec, 'recall':recl, 'f1':f1})
    return result

# What's the problem?

* Informal: using Telstra's service logs, the task is to predict the severity of a network disruption.

* Describe the problem formally, e.g.
    * Task (T): For a specific time and location, classify network fault severity as 0 == no faults, 1 == few faults, 2 == many faults. Fault severity is a measurement of actual reported faults from users of the network and is the target variable.
    * Experience (E): Each row in the train/test data represents a location and a time point. They are identified by the "id" column, which is the key "id" used in the other data files (event_type, resource_type, severity_type, log_feature) that contain features extracted from log files and other sources. Note that severity_type:
        * Is a categorical feature extracted from the log files
        * "Often" (**when is it not?**) the type of the warning message coming from the log files
        * Does not have an ordering
    * Performance (P): Classification accuracy, the number of fault events predicted correctly (a set of predicted probabilities one for every fault severity mus be given) out of all faults considered as a percentage.

* Assumptions:
    * All given features will matter to the model.
    * The clustering in time and location matter to the model, but may be mutually exclusive.
    * The volume of faults may not matter to the model.
    * Systemic long duration, low-volume correlated faults could be hiding in data.

* Similar problems:
    * Airline loyalty and medallion class problem
    * Anomaly detection, e.g. [Twitter's R package](https://github.com/nicolasmiller/pyculiarity)

# How to approach the problem?

* Brainstorm! Literally type everything as a stream of consciousness (coffee helps here)

These are my thoughts. These are my thoughts. These are my thoughts. These are my thoughts. These are my thoughts. These are my thoughts. These are my thoughts. These are my thoughts. These are my thoughts. These are my thoughts.

* Describe how to solve problem manually
* What data do I have?
* What data do I need?
* What data I **don't** need?
* What do I know the least about in the solution process above?
    * knowledge gap 1
    * knowledge gap 2
* What am I concerned is incorrect in above solution? Find an expert online and ask them about these items
    * concern 1
    * concern 2

# Data Preparation

## Summarize

In [ ]:
# go through each file and tell me what's in it
for f in files:
    df = pd.read_csv(f)
    cols = df.columns.tolist()
    obs = df.shape[0]
    print '{:15} {:s}'.format('Filename:', f)
    print '{:15} {:d}'.format('Observations:', obs)
    for c in cols:
        uflag = ""
        nflag = ""
        uni = len(pd.unique(df[c].ravel()))
        nulls = np.count_nonzero(df[c].isnull())
        if obs != uni:
            uflag = '*MULTIPLE VALS PER KEY*'
        if nulls >0 :
            nflag = '*NULLS IN COL*'
        print '{:15} {:d} {:20} {:20}'.format(c, uni, uflag, nflag)
    print '\n'

* Column names look fine, don't need to clean those
* No nulls, hooray
* Looks like each ID has only one severity type and multiple event types| log features| resource types:

For example, in the training set, consider ID 10005:

|id| item | code | sev/vol
|:-|:-|:-:|:-:
|10005 | location |802 | 0
|10005|severity_type |1|
|10005|event_type |11|
|10005|event_type |13|
|10005|event_type |14|
|10005|log_feature| 345|1
|10005|log_feature| 54|1
|10005|resource_type |4|
|10005|resource_type |6|
|10005|resource_type |8|

Transposing would be

|id|loc|sev|evt|fea|res
|-|
|10005|{802:0}|1|[11,13,14]|{54:1,345:1}|[4,6,8]

For a single id, the coding can't be a list, so one hot encode every type? How many are we talking?
* 53 event types (binary)
* 386 log features (continuous)
* 10 resource types (binary)

That's 449 features. Regardless, need to join all this data together. It's all categorical data (code_n is y/n), except for the log features which has a volume.

## Basics

* Replace missing values
* Remove duplicates
* One-Hot encode categorical features
* Find outliers and explain
* Scale
* Standardize

In [ ]:
# read raw volumes
df = pd.read_csv(files[1])

# cleanup col names
for c in df.columns:
    if df[c].dtypes == 'object':
        df[c] = df[c].map(lambda x: re.sub("[^0-9]", "", x))
        
# re-org by id        
vols = df.pivot(index='id', columns='log_feature', values='volume').fillna(0)
cnames = ['log_feature_'+str(x) for x in vols.columns]

# scale or min-max
vols_scl = vols.apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x))
vols_scl.columns = cnames

In [ ]:
# build train df
df = pd.read_csv(files[5], index_col=0)
for c in df.columns:
    if df[c].dtypes == 'object':
        df[c] = df[c].map(lambda x: re.sub("[^0-9]", "", x))
df_train = df.copy()

# build test df
df = pd.read_csv(files[4], index_col=0)
for c in df.columns:
    if df[c].dtypes == 'object':
        df[c] = df[c].map(lambda x: re.sub("[^0-9]", "", x))
df_test = df.copy()

print df_train.shape
print df_test.shape

In [ ]:
# add categorical event_type, resource_type, severity_type
nfiles = [files[0], files[2], files[3]]
for f in nfiles:
    df = pd.read_csv(f, index_col=0)
    for c in df.columns:
        df[c] = df[c].map(lambda x: re.sub("[^0-9]", "", x))
    dfv = pd.get_dummies(df).groupby(df.index).sum()
    df_train = df_train.join(dfv)
    df_test = df_test.join(dfv)

In [ ]:
# join scaled volumes
df_train = df_train.join(vols_scl)
df_test = df_test.join(vols_scl)

In [ ]:
# gut check the 10005 entry from above
junk = df_train.T
x = junk.loc[:, 10005] > 0
x[x].index

In [ ]:
# one time analysis to re-bin volunme data
#df = pd.read_csv(files[1], index_col=0)
#x = np.log10(df['volume'][(df.volume > 0)].values)
#sns.distplot(x, bins=8, kde=False, fit=stats.expon)

# grab the bins in log space use the built-in Freedman-Diaconis rule
#hist, bins = np.histogram(x, bins=8)

# assign each volume to a bin
#inds = np.digitize(x, bins)

# add this to the df
#df['volume'] = inds

## Save

In [ ]:
# merge df's
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)
df_train['is_train'] = True
df_test['is_train'] = False
df_all = pd.concat([df_train, df_test], axis=0, ignore_index=False)
a = df_train.shape[0]
b = df_test.shape[0]
c = df_all.shape[0]
assert(a + b == c, "DF's are not summing correctly")

In [ ]:
# save to hdf5 for easier loading later
call(["rm", "-rf", hdf_file])
hdf = pd.HDFStore(hdf_file)
hdf.put('df_test', df_test)
hdf.put('df_train', df_train)
hdf.put('df_all', df_all)
hdf.close()
call(["lrztar", "-zf", hdf_file])

# Reload Point

In [49]:
# get datadir
import platform
uname = platform.uname()[0]
if uname == 'Linux':
    datadir = '/home/kcavagnolo/ml_fun/telstra/data/'
elif uname == 'Darwin':
    datadir = '/Users/cavagnolo/ml_fun/telstra/data/'
else:
    raise OSError("Unknown system: "+str(uname))

# get files
files = sorted(glob.glob(datadir+'*.csv'))
hdf_file = datadir + 'features.h5'

# reopen hdf store
hdf = pd.HDFStore(hdf_file)

# reload df's
print hdf
df_test = hdf['df_test']
df_train = hdf['df_train']
df_all = hdf['df_all']
#df_svd = hdf['df_svd']
#df_pca = hdf['df_pca']
#df_kpca = hdf['df_kpca']
#df_lle = hdf['df_lle']
#df_tsne = hdf['df_tsne']
hdf.close()

# relevant columns
#svdcols = [c for c in df_svd.columns if c[:3] == 'svd']
#pcacols = [c for c in df_pca.columns if c[:3] == 'pca']
#kpcacols = [c for c in df_kpca.columns if c[:4] == 'kpca']
#llecols = [c for c in df_lle.columns if c[:3] == 'lle']

# cols to drop
attribute_drop = ['fault_severity', 'id', 'location', 'is_train']

# col prefixes
attributes = ['event_', 'resource_', 'severity_', 'log_feature_']

<class 'pandas.io.pytables.HDFStore'>
File path: /home/kcavagnolo/ml_fun/telstra/data/features.h5
/df_all              frame        (shape->[18552,458])
/df_test             frame        (shape->[11171,457])
/df_train            frame        (shape->[7381,458]) 


# Attribute Summaries

In [ ]:
df_train.filter(regex='log_feature_').mean().order(ascending=False).head()

In [ ]:
df_all['log_feature_312'][(df_all.log_feature_312 > 0)].count().astype(float)/len(df_all)

## Histograms

In [ ]:
# items to plot
a = ['event_', 'resource_', 'severity_', 'log_fea']

# build axes
mpl_fig = plt.figure()
ax1 = mpl_fig.add_subplot(411)
ax2 = mpl_fig.add_subplot(412)
ax3 = mpl_fig.add_subplot(413)
ax4 = mpl_fig.add_subplot(414)
axs = [ax1, ax2, ax3, ax4]

# iterate over each
for cond in [True, False]:
    for i, b in enumerate(a):
        y = df_all[(df_all.is_train == cond)].filter(regex=b).apply(pd.value_counts).fillna(0).T[0]
        y = 100.*(1.0-(y/df_all[(df_all.is_train == cond)].shape[0]))
        x = range(1, len(y)+1)
        axs[i].bar(x, y, label=a[i])
        axs[i].set_ylabel(a[i])
py.iplot_mpl(mpl_fig, strip_style=True)

Distributions looks similar, so train will cv w/ test. Any structure in sequence of ID's?

## Sequencing

In [ ]:
fault = df_all[(df_all.is_train == True)].fault_severity.astype(float)
n = len(fault)
plt.bar(np.arange(n), fault)
plt.show()

In [ ]:
loc = df_train.location.astype(float)
plt.bar(np.arange(n), loc)
plt.show()

Both look like noise to me, gonna leave it for now. What about correlations among the various codes?

## Correlations

In [ ]:
a = ['severity_type', 'event_type', 'resource_type']#, 'log_feature']
for b in a:

    # correlation matrix
    c = df_all.filter(regex=b).copy()
    corr = c.corr(min_periods=len(c.columns)/10)

    # colormap
    cmap = sns.light_palette("navy", as_cmap=True, reverse=True)
    
    # clustered heatmap of distances w/o mask
    # dist 0 --> close, 1 --> distant
    dsim = 1.0-np.abs(corr)
    
    # find degenerate params
    mask = np.ones(dsim.shape,dtype='bool')
    mask[np.triu_indices(len(dsim))] = False
    x = ((dsim < 0.2) & mask).values.nonzero()
    a = zip(x[0], x[1])
    for x, y in a:
        print dsim.index[x], dsim.columns[y]

    # plot matrix
    if len(dsim) > 10:
        annot=False
    else:
        annot=True
    plt.figure(figsize=(15, 12))
    g = sns.clustermap(data=dsim,
                       method='complete',
                       metric='correlation',
                       cmap=cmap, linewidths=0.5, vmin=0.0, vmax = 1.0,
                       annot=annot, fmt='.2f', annot_kws={'size':'10'})
    plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)

In [ ]:
# log_feature specific
c = df_all.filter(regex='log_feature').copy()
log_corr = c.corr(min_periods=len(c.columns)/10)
log_dsim = 1.0-np.abs(log_corr)

In [ ]:
cmap = sns.light_palette("navy", as_cmap=True, reverse=True)
g = sns.clustermap(data=log_dsim,
                   method='complete',
                   metric='correlation',
                   cmap=cmap,
                   xticklabels=False, yticklabels=False,
                   linewidths=0.0,
                   vmin=0.0,
                   vmax = 1.0)

The serverity types are mostly anti-correlated except type 1 and 2.

There is interesting structure in the resource and event types. Makes me more curious about how these events are networked, i.e. communicating to each other? Build out as network problem? Can't: id and loc are 1:1 so the only connections will be among events that are the same, in that they have the same coding across all types. I don't see that as informative to a model.

Log features are also clustered.

**But, this is clearly a well-defined classification problem.**

# Baseline Models

In [ ]:
# train data
X_train = df_all[(df_all.is_train==True)].copy().drop(drop, 1)
xcols   = X_train.columns
X_train = X_train.values
y_train = df_all[(df_all.is_train==True)].fault_severity.values

# random test, should be close to 50%
pred_rand = np.random.choice(y_train, len(y_train))
print 'Random', metrics.accuracy_score(y_train, pred_rand)

# couple models
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.ensemble import BaggingClassifier as BC
from sklearn.ensemble import RandomForestClassifier as RF

# classifiers
clfs = []
clfs.append(('DecTree', DecisionTreeClassifier(max_depth=30)))
clfs.append(('Bagging', BaggingClassifier(n_estimators=100, n_jobs=-1)))
clfs.append(('RandFor', RandomForestClassifier(n_estimators=3000, n_jobs=-1)))

# score each on cv
for clf in clfs:
    scores = cross_validation.cross_val_score(clf[1], X_train, y_train, cv=5)
    print clf[0], scores.mean(), '+/-', scores.std()

In [ ]:
results = run_cv(df_train, attributes, DT, **kwargs)
plt.figure(figsize=(16, 6))
ax = plt.subplot(111)
results.loc[:, 'accuracy'].plot(kind='line', ax=ax, rot=90, marker='o')
results.loc[:, 'precision'].plot(kind='line', ax=ax, color='r', rot=90, marker='o')
#results.loc[:, 'recall'].plot(kind='line', ax=ax, color='y', rot=90, marker='o')
results.loc[:, 'f1'].plot(kind='line', ax=ax, color='g', rot=90, marker='o')
plt.xticks(range(len(results.index)), results.attr_combo)
plt.show()

In [ ]:
# fit a model
clf = clfs[2]
clf = pipeline.Pipeline([('scaler', preprocessing.StandardScaler()), ('clf', clf)])
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_train)
kscore(y_pred, y_train)

In [ ]:
X_test = df_all[(df_all.is_train==False)].copy().drop(drop, 1).values
ids = df_all[(df_all.is_train==False)].id.values
prob = clf.predict_proba(X_test)

df_result = pd.DataFrame({
        'id': ids,
        'predict_0': prob[:, 0],
        'predict_1': prob[:, 1],
        'predict_2': prob[:, 2]})
df_result.to_csv('first_submission.csv', index=False)

# Feature Engineering

* Data has complex multi-dimensional structure that ML algos know how to find and exploit to make decisions
* What's not exposed to the models?
    * Events are too dense or too complex for the algo to find. Fix == dim reduction (PCA or SVD)
    * How the various events are communicating. Fix == clustering
    * There is more attribute aggregation/creation needed. Fix == hyperattributes
    * Time and location dependance of the events. Fix == ???

* Remove correlated features
* Remove features using statistical tests
* Try pair-wise feature interactions, e.g. a*b, a-b, a+b, a/b
* Try feature transformations, e.g. sqrt(a), log(a), abs(a)

## Importance

In [ ]:
# train data
X_train = df_all[(df_all.is_train==True)].copy().drop(drop, 1)
xcols   = X_train.columns
X_train = X_train.values
y_train = df_all[(df_all.is_train==True)].fault_severity.values

# simple rand forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=3000, n_jobs=-1)
scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=15, n_jobs=-1)
print scores.mean(), '+/-', scores.std()

# top ten features
clf = pipeline.Pipeline([('scaler', preprocessing.StandardScaler()), ('clf', clf)])
clf.fit(X_train, y_train)
fea_impt = zip(xcols, (clf.feature_importances_ * 100.0).astype(int))
sorted(fea_impt, key=lambda x: -x[1])[0:10]

## Extraction

## Selection

### Univariate

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectPercentile, f_classif

X = df_train.copy().drop(drop, 1)
cols = X.columns
X = X.values
y = df_train.fault_severity.values.astype(int)

mpl_fig = plt.figure(1, figsize=(11,8))
plt.clf()

X_indices = np.arange(X.shape[-1])

# Univariate feature selection with F-test for feature scoring
selector = SelectPercentile(f_classif, percentile=50)
selector.fit(X, y)

# get scores
scores = -np.log10(np.array(selector.pvalues_))
scores[np.isnan(scores)] = 0
scores /= scores.max()
plt.bar(X_indices - .45, scores,
        width=.2, label=r'Univariate score ($-Log(p_{value})$)', color='g')

# Compare to the weights of an SVM
clf = LinearSVC(C=0.3, penalty="l1", dual=False)
clf.fit(X, y)

svm_weights = (clf.coef_ ** 2).sum(axis=0)
svm_weights /= svm_weights.max()

plt.bar(X_indices - .25, svm_weights,
        width=.2, label='SVM weight', color='r')

clf_selected = LinearSVC(C=0.3, penalty="l1", dual=False)
clf_selected.fit(selector.transform(X), y)

svm_weights_selected = (clf_selected.coef_ ** 2).sum(axis=0)
svm_weights_selected /= svm_weights_selected.max()

plt.bar(X_indices[selector.get_support()] - .05, svm_weights_selected,
        width=.2, label='SVM weights after selection', color='b')


plt.xlabel('Supported feature number')
plt.yticks(())
plt.axis('tight')
plt.legend(loc='upper right')
plt.show()

In [ ]:
print selector.transform(X).shape[1]
uni_fea = Xcols[selector.get_support(indices=True)].tolist()

In [ ]:
# get index of max value
idx = X_indices[selector.get_support()][svm_weights_selected.argmax()]

# get column name
Xcols[idx]

Well no shit, it's log_feature_203 again. Check the baseline models section, it showed up in the decision tree feature importance list as #1 and being almost 2x more important than #2.

### Recursive

Want to know what features may be the most important before any transformation. Try recursive feature elimination with cross-validation (RFECV). For multiclass, stratified K-fold used by default and shuffle is True.

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import SGDClassifier as SGD
from sklearn.naive_bayes import MultinomialNB as MNB

# get training data
X = df_all[(df_all.is_train == True)].copy()
y = X.fault_severity
X.drop(drop, 1, inplace=True)

# make df to store results
df_frank = pd.DataFrame({'feature':X.columns.values})

# create base classifier
models =[]
models.append(LR(n_jobs=-1))
models.append(RF(n_jobs=-1))
models.append(SGD(n_jobs=-1))
models.append(MNB())

# iterate over models
for m in models:
    mname = str(m).split('(')[0]
    rfecv = RFECV(m, cv=3, scoring='accuracy')
    rfecv = rfecv.fit(X, y)
    print("%s Optimal # features: %d" % (mname, rfecv.n_features_))
    c = 'rfecv_'+str(mname[:3])
    df_frank[c] = rfecv.ranking_
    
    # n_features vs cv score
    plt.figure()
    plt.title(mname)
    plt.xlabel("# features")
    plt.ylabel("CV score")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.axvline(rfecv.n_features_, color='r', linestyle='--')
    plt.show()

MNB is not a good model, nor SGD. Try SVC linear in their place.

In [ ]:
from sklearn.svm import SVC
m = SVC(kernel='linear')
mname = str(m).split('(')[0]
rfecv = RFECV(m, cv=3, scoring='accuracy')
rfecv = rfecv.fit(X, y)
print("%s Optimal # features: %d" % (mname, rfecv.n_features_))
c = 'rfecv_'+str(mname[:3])
df_frank[c] = rfecv.ranking_
    
# n_features vs cv score
plt.figure()
plt.title(mname)
plt.xlabel("# features")
plt.ylabel("CV score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.axvline(rfecv.n_features_, color='r', linestyle='--')
plt.show()

In [ ]:
# get avg rank and abs rank
df_frank['avg_rank'] = df_frank[['rfecv_Log', 'rfecv_Ran', 'rfecv_SVC']].mean(axis=1)
df_frank['abs_rank'] = df_frank.avg_rank.rank(method='min')

In [ ]:
# save to hdf5 file
hdf = pd.HDFStore(hdf_file)
hdf.put('df_frank', df_frank)
hdf.close()
call(["lrztar", "-zf", hdf_file])

### Weights

In [ ]:
from sklearn.svm import LinearSVC
X = df_train.copy().drop(drop, 1)
cols = X.columns
y = df_train.fault_severity
clf = LinearSVC(C=0.3, penalty="l1", dual=False)
selector = feature_selection.SelectFromModel(clf, threshold=0.50)
selector.fit(X, y)

In [ ]:
print selector.transform(X).shape[1]
imp_fea = cols[selector.get_support(indices=True)].tolist()

## Construction

### Dimension Reduction

The correlation matricies show there are redundant values. How many components in the full dataset to preserve? The eigenvalues in SVD should help determine what attributes are valuable. Data is sparse matrix of binary and multiclass values, options for analysis:
* [Single value decomposition and Principal component analysis](http://blog.applied.ai/visualising-high-dimensional-data/)
* [Kernel PCA](http://sebastianraschka.com/Articles/2014_kernel_pca.html)
* [Locally Linear Embedding](http://scikit-learn.org/stable/auto_examples/manifold/plot_swissroll.html)
* [t-Distributed Stochastic Neighbor Embedding](https://www.oreilly.com/learning/an-illustrated-introduction-to-the-t-sne-algorithm)

#### SVD

In [ ]:
# get rid of the cols that I suspect are not important
X = df_all.copy().drop(drop, 1)

# orig num comps
ncomps = X.shape[1] - 1

# run svd
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=ncomps)
svd_fit = svd.fit(X)

# print out variance ranges
for i in range(0, X.shape[1], 20):
    svar = svd_fit.explained_variance_ratio_.cumsum()[i]
    if svar >= 0.95:
        print('Variance preserved at {:} components == {:.1%}'.format(i, svar))

# plot result        
ax = pd.Series(svd_fit.explained_variance_ratio_.cumsum()).plot(kind='line')

At 60 components 98% of variance is preserved, so try keeping just those in a new df.

In [ ]:
# transform X
ncomps = 60
X_svd = TruncatedSVD(n_components=ncomps).fit_transform(X)

# save svd to df and add back dropped cols
df_svd = pd.DataFrame(X_svd, columns=['svd{}'.format(c) for c in range(ncomps)], index=df_all.index)
for c in drop:
    df_svd[c] = df_all[c]

# save col names for easy filter later
svdcols = [c for c in df_svd.columns if c[:3] == 'svd']

# check size
assert(df_svd.shape[0] == df_all.shape[0])

In [ ]:
# save to hdf5 file
hdf = pd.HDFStore(hdf_file)
hdf.put('df_svd', df_svd)
hdf.close()
call(["lrztar", "-zf", hdf_file])

#### Linear PCA

In [ ]:
# get rid of the cols that I suspect are not important
X = df_all.copy().drop(drop, 1)

# orig num comps
ncomps = X.shape[1] - 1

# run pca
from sklearn.decomposition import PCA, KernelPCA
pca = PCA(n_components=ncomps, whiten=True)
pca_fit = pca.fit(X)

# plot result        
ax = pd.Series(pca_fit.explained_variance_ratio_.cumsum()).plot(kind='line')

In [ ]:
# transform X
ncomps = 2
X_pca = PCA(n_components=ncomps, whiten=True).fit_transform(X)

# check transform
print "Mean: ", np.round(X_pca.mean(axis=0), decimals=5)
print "Stdv: ", np.round(X_pca.std(axis=0), decimals=5)
print "Corr: ", np.round(np.corrcoef(X_pca.T), decimals=5)

# save pca to df and add back dropped cols
df_pca = pd.DataFrame(X_pca,
                      columns=['pca{}'.format(c) for c in range(ncomps)],
                      index=df_all.index)
for c in drop:
    df_pca[c] = df_all[c]

# save col names for easy filter later
pcacols = [c for c in df_pca.columns if c[:3] == 'pca']

# check size
assert(df_pca.shape[0] == df_all.shape[0])

In [ ]:
g = sns.lmplot('pca0', 'pca1',
               data=df_pca,
               fit_reg=False,
               col="fault_severity")

In [ ]:
# save to hdf5 file
hdf = pd.HDFStore(hdf_file)
hdf.put('df_pca', df_pca)
hdf.close()
call(["lrztar", "-zf", hdf_file])

Doesn't look like linearly separable data (as exposed in correlation tasks above). Need to try kernel PCA to remove the nonlinearity.

#### Kernel PCA

In [ ]:
# transform X
X_kpca = KernelPCA(n_components=2, kernel="rbf", gamma=20).fit_transform(X)

In [ ]:
# save pca to df and add back dropped cols
df_kpca = pd.DataFrame(X_kpca,
                       columns=['kpca{}'.format(c) for c in range(ncomps)],
                       index=df_all.index)

for c in drop:
    df_kpca[c] = df_all[c]

# save col names for easy filter later
kpcacols = [c for c in df_kpca.columns if c[:4] == 'kpca']

# check size
assert(df_kpca.shape[0] == df_all.shape[0])

In [ ]:
g = sns.lmplot('kpca0', 'kpca1',
               data=df_kpca,
               fit_reg=False,
               col="fault_severity")

In [ ]:
# save to hdf5 file
hdf = pd.HDFStore(hdf_file)
hdf.put('df_kpca', df_kpca)
hdf.close()
call(["lrztar", "-zf", hdf_file])

#### LLE

In [ ]:
from sklearn import manifold
X_lle, err = manifold.locally_linear_embedding(X, n_neighbors=12,
                                             n_components=2)
print("Done. Reconstruction error: %g" % err)

In [ ]:
# save pca to df and add back dropped cols
df_lle = pd.DataFrame(X_lle,
                      columns=['lle{}'.format(c) for c in range(ncomps)],
                      index=df_all.index)
for c in drop:
    df_lle[c] = df_all[c]

# save col names for easy filter later
llecols = [c for c in df_lle.columns if c[:3] == 'lle']

# check size
assert(df_lle.shape[0] == df_all.shape[0])

In [ ]:
g = sns.lmplot('lle0', 'lle1',
               data=df_lle,
               fit_reg=False,
               col="fault_severity")

In [ ]:
# save to hdf5 file
hdf = pd.HDFStore(hdf_file)
hdf.put('df_lle', df_lle)
hdf.close()
call(["lrztar", "-zf", hdf_file])

#### t-SNE

Tried SVD and KPCA inputs: SVD outperformed any other. Using it as input to t-SNE

In [ ]:
# build tsne fitter
from sklearn.manifold import TSNE
#metric = metrics.pairwise.manhattan_distances
tsne = TSNE(n_components=2,
            #init='pca',
            perplexity=50,
            #metric=metric,
            learning_rate=500,
            method='barnes_hut',
            verbose=2)

# fit training data only
Z = tsne.fit_transform(df_svd[svdcols])

# save to df
df_tsne = pd.DataFrame(Z, columns=['x','y'], index=df_svd.index)
for c in drop:
    df_tsne[c] = df_svd[c]

# check size
assert(df_tsne.shape[0] == df_svd.shape[0])

In [ ]:
# kpca
g = sns.lmplot('x', 'y',
               data=df_tsne,
               fit_reg=False,
               col="fault_severity")

In [ ]:
# save to hdf5 file
hdf = pd.HDFStore(hdf_file)
hdf.put('df_tsne', df_tsne)
hdf.close()
call(["lrztar", "-zf", hdf_file])

### Clustering

Clustering may be interesting too: could add cluster membership as a new feature. 

* [Binary Jaccard similarity matrix fed into hierarchical cluster and then using the top "nodes"](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_similarity_score.html)
* [K-mode/median clustering](https://github.com/nicodv/kmodes)
* [Cosine similarities fed into spectral clustering or dbscan](http://stackoverflow.com/questions/30089675/clustering-cosine-similarity-matrix)
* Frequent itemset mining

#### DBScan

In [ ]:
# remove the log_feature features which are scaled volumes
ecols = [c for c in df_all.columns if 'log_feature' not in c]
X = df_all.copy()[ecols]
X = X[(X.is_train == True)].drop(drop, 1)

In [ ]:
# base similarity matrix
similarity = np.dot(X, X.T)

# squared magnitude of preference vectors (number of occurrences)
square_mag = np.diag(similarity)

# inverse squared magnitude
inv_square_mag = 1 / square_mag

# if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
inv_square_mag[np.isinf(inv_square_mag)] = 0

# inverse of the magnitude
inv_mag = np.sqrt(inv_square_mag)

# cosine similarity
cosine = similarity * inv_mag
cosine = cosine.T * inv_mag

In [ ]:
# find whole factors for matrix
factors(cosine.shape[0])

In [ ]:
# plot it
binning = 11
plt.figure(figsize=(14, 11))
cmap = sns.light_palette("blue", as_cmap=True)
plt.imshow(cosine[::binning, ::binning], interpolation='none', cmap=cmap)

In [ ]:
# dbscan expects distances, so need 1-cosine input
from sklearn.cluster import DBSCAN
dbclust = DBSCAN(min_samples=30, eps=0.6).fit_predict(1.0-cosine)

In [ ]:
# viz the cluster dist
from collections import Counter
for i in Counter(dbclust).items():
    print i
sns.distplot(dbclust, kde=False)

#### K-modes

In [ ]:
from kmodes import kmodes

# build the clusters
km = kmodes.KModes(n_clusters=22, init='Cao', n_init=5, verbose=1)
clusters = km.fit_predict(X)

# viz the cluster dist
for i in Counter(clusters).items():
    print i
sns.distplot(clusters, kde=False)

That's not a terrible result, it somewhat mimics the tsne result, except for bins 3 and 4, as I don't see any isolated points in tsne, but it's not a 1:1 comparison. Let's see how this performs. First add the cluster assignments to the X matrix.

#### Hierarchical

revisit the correlation measures
run a hierarch model

In [ ]:
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.cluster.hierarchy import fcluster

# create dist matrix
Y = 1.0 - np.abs(df_svd.corr())
Z = linkage(Y, 'complete', 'correlation')

# check cophenetic coeffs; closer to 1, the better
c, coph_dists = cophenet(Z, Y)
print c

# view dendro
dendro = dendrogram(z, labels=Y.columns)
plt.show()

In [ ]:
k = 2
clusters = fcluster(Z, k, criterion='maxclust')
plt.figure(figsize=(14, 11))
plt.scatter(df_svd[:, 0], df_svd[:, 1], c=clusters, cmap='prism')
plt.show()

Check validity of clusters using permutation tests

In [ ]:
# create permutation
svm = SVC(kernel='linear')
cv = StratifiedKFold(y, 2)
score, permutation_scores, pvalue = permutation_test_score(
    svm, X, y, scoring="accuracy", cv=cv, n_permutations=100, n_jobs=-1)
print("Classification score %s (pvalue : %s)" % (score, pvalue))

# histo of permu scores
plt.hist(permutation_scores, 20, label='Permutation scores')
ylim = plt.ylim()
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score (pvalue %s)' % pvalue)
plt.plot(2 * [1. / n_classes], ylim, '--k', linewidth=3, label='Luck')
plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.show()

In [ ]:
# append cluster membership to the no-location version of X
a = np.array(clusters)[np.newaxis]
print 'X no loc ', X_nl.shape
X_nlc = np.concatenate((X_nl, a.T), axis=1)
print 'X no loc+clu ', X_nlc.shape

## Learning

### Stacking

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

X = 
y = 

# pca
pca = PCA(n_components=2)

# univariate
selection = SelectKBest(k=1)

# combined estimator
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)

svm = SVC(kernel="linear")

# Do grid search over k, n_components and C:
pipeline = Pipeline([("features", combined_features), ("svm", svm)])
param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[1, 2],
                  svm__C=[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
grid_search.fit(X, y)
print(grid_search.best_estimator_)

# Model Evaluation

## Lasagne (Theano)

* use PCA with NN (2 hidden layers + ReLU + dropout)
* shooting for logloss < 0.5

In [ ]:
import lasagne

# real train data
X = df_all[(df_all.is_train==True)].copy()
y = X.fault_severity.values
X.drop(drop, 1, inplace=True)
X = X.as_matrix()

# theano format
X = X.astype(thno.config.floatX)
y = y.astype('int32')

In [ ]:
# input layer
l_in = lasagne.layers.InputLayer(
    shape=X.shape
)

# hidden layer1
l_hidden1 = lasagne.layers.DenseLayer(
    l_in,
    num_units=30,
    nonlinearity=lasagne.nonlinearities.rectify,
    W=lasagne.init.HeNormal(gain='relu')
)

# hidden layer2
l_hidden2 = lasagne.layers.DenseLayer(
    l_hidden1,
    num_units=10,
    nonlinearity=lasagne.nonlinearities.rectify,
    W=lasagne.init.HeNormal(gain='relu')
)

# dropout layer
l_dropout = lasagne.layers.DropoutLayer(
    l_hidden2,
    p=0.25
)

# output layer
l_output = lasagne.layers.DenseLayer(
    l_dropout,
    num_units=3,
    nonlinearity=lasagne.nonlinearities.softmax)

In [ ]:
output_train = lasagne.layers.get_output(l_output, X)

loss_train = lasagne.objectives.aggregate(
    lasagne.objectives.categorical_crossentropy(output_train, y))

output_eval = lasagne.layers.get_output(l_output, X, deterministic=True)

loss_train = lasagne.objectives.aggregate(
    lasagne.objectives.categorical_crossentropy(output_eval, y))

In [ ]:
all_params = lasagne.layers.get_all_params(l_output)

In [ ]:
# Use ADADELTA for updates
updates = lasagne.updates.adadelta(loss_train, all_params)
true_output = T.ivector('true_output')
train = theano.function([l_in.input_var, true_output], loss_train, updates=updates)

In [ ]:
# train in mini-batches
batch_size = 100
n_epochs = 10

# iterate
batch_idx = 0
epoch = 0
while epoch < n_epochs:
    # extract train data/label batch and update the parameters with it
    train(dataset['train']['X'][batch_idx:batch_idx + batch_size],
 
          
          
          dataset['train']['y'][batch_idx:batch_idx + BATCH_SIZE])
 batch_idx += BATCH_SIZE
 # Once we've trained on the entire training set...
 if batch_idx >= dataset['train']['X'].shape[0]:
 # Reset the batch index
 batch_idx = 0
 # Update the number of epochs trained
 epoch += 1
 # Compute the network's on the validation data
 val_output = get_output(dataset['valid']['X'])
 # The predicted class is just the index of the largest probability in the output
 val_predictions = np.argmax(val_output, axis=1)
 # The accuracy is the average number of correct predictions
 accuracy = np.mean(val_predictions == dataset['valid']['y'])
 print("Epoch {} validation accuracy: {}".format(epoch, accuracy))

## Keras (Theano)

In [ ]:
# quick existing model test
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
thno.config.compute_test_value = 'warn'

# base data
X_train = df_train.copy().drop(drop, 1).as_matrix()
y_train = df_train.fault_severity.values

In [ ]:
# rebuild the train and test sets
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.4)
Y_train = np_utils.to_categorical(y_train, 3)
Y_test = np_utils.to_categorical(y_test, 3)

# common vars
nb_epoch = 100
dims = X_train.shape[1]
fs = list(factors(dims))
batch_size = fs[-1]/4

# NN model and layers
model = Sequential()
model.add(Dense(512, input_shape=(dims,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))

# build model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# fit the model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          show_accuracy=True,
          verbose=1,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test,
                       show_accuracy=True,
                       verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# base data
X = df_test.copy()
ids = X.id
for c in drop:
    if c in X.columns:
        X.drop(c, 1, inplace=True)
X = X.as_matrix()
y_probs = model.predict(X)

final = [[iden, (y_probs[j][0]), (y_probs[j][1]), (y_probs[j][2])] for j, iden in enumerate(ids)]
frame = pd.DataFrame(final,columns=('id','predict_0','predict_1','predict_2'))
frame.to_csv("keras.csv",index=False)

|Run|Local  |LB     |Changes|
|---|-------|-------|:------|
|1  |0.62779|0.65748|Initial
|2  |0.67230|-------|Adadelta opt
|3  |0.75352|-------|SGD opt
|4  |0.82378|-------|Adam


According to forums:
"The norm for CV and LB disagreement seems to be about 0.01-0.03 based on what I've experienced and read from others. A disagreement as large as you have indicates you're probably over-fitting and may have some minor leakage."

## XGBoost

In [ ]:
# get train inputs
X_train = df_train.copy()
y_train = X_train.fault_severity
X_train.drop(drop, 1, inplace=True)

# get test inputs
X_test = df_test.copy()
X_test.drop(drop, 1, inplace=True)

# split train into... train and test
from sklearn.cross_validation import train_test_split
xg_train, xg_test, yg_train, yg_test = train_test_split(X_train, y_train, test_size=0.33)

In [ ]:
import xgboost as xgb

xg_train = xgb.DMatrix(xg_train, label=yg_train.values)
xg_test = xgb.DMatrix(xg_test, label=yg_test.values)

In [ ]:
xg_train.save_binary("xg_train.buffer")
xg_test.save_binary("xg_test.buffer")

In [ ]:
nclass = 3
param = {'booster':'gbtree',
         'objective': 'multi:softprob',
         'num_class': nclass,
         'eval_metric': 'mlogloss',
         'max_depth': 0,
         'subsample': 0.9,
         'colsample_bytree': 0.9,
         'eta': 0.0,
         'gamma': 0.0,
         #'alpha': 0.1,
         #'lambda': 0.1,
         'silent': True}

In [ ]:
clf = MyXGBClassifier(n_rounds=n_rounds,eta=0.2,max_depth=10,subsample=0.9,colsample_bytree=0.9)
gridsearch = GridSearchCV(clf, params, scoring=scorer_logloss, n_jobs=10, cv=3, refit=True)
gridsearch.fit(X,Y)

grid_scores = gridsearch.grid_scores_
top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:3]
for i, score in enumerate(top_scores):
    print("Model with rank: {0}".format(i + 1))
    print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
            score.mean_validation_score,
            np.std(score.cv_validation_scores)))
    print("Parameters: {0}".format(score.parameters))

In [ ]:
num_round = 1000
best_perr = 1.0
for gamma in (0, 0.5, 1):
    for depth in range(5, 20, 5):
        for eta in np.arange(0.1, 0.22, 0.02):
            param['max_depth'] = depth
            param['eta'] = eta
            param['gamma'] = gamma
            bst = xgb.train(param, xg_train, num_round)
            yg_prob = bst.predict(xg_test).reshape(yg_test.shape[0], nclass)
            yg_label = np.argmax(yg_prob, axis=1)
            perr = sum(int(yg_label[i]) != yg_test.values[i] for i in
                       range(len(yg_test.values))) / float(len(yg_test.values))
            print "Gamma: %.3f, Depth: %d, Eta: %.3f, Err: %.3f" % (gamma, depth, eta, perr)
            if perr < best_perr:
                print '**** NEW BEST *****'
                best_perr = perr
                best_depth = depth
                best_eta = eta
                best_gamma = gamma
print "Best Pred. %.3f (Depth %d, Eta %.3f, Gamma %.3f)" % (best_perr, best_depth, best_eta, best_gamma)
param['max_depth'] = best_depth
param['eta'] = best_eta
param['gamma'] = best_gamma

In [ ]:
# retrain best model
bst = xgb.train(param, xg_train, num_round)

In [ ]:
bst.save_model('0001.model')
bst.dump_model('dump.raw.txt')

In [ ]:
# plot feature importance
f, ax = plt.subplots(figsize=(8,20))
xgb.plot_importance(bst, ax=ax)

In [ ]:
# train on full set
X_train = xgb.DMatrix(X_train, label=y_train.values)
bst = xgb.train(param, X_train, num_round)

In [ ]:
# setup test set
X_test = xgb.DMatrix(X_test)
y_pred = bst.predict(X_test)

In [ ]:
dtest = df_all[(df_all.is_train == False)].copy()
with open("submission.csv", "w") as f:
    f.write("id,predict_0,predict_1,predict_2\n")
    for i, item in enumerate(ypred):
        f.write(str(int(dtest.id.iloc[i])) + "," + ",".join(map(lambda x:str(x), item)) + "\n")

In [ ]:
x_train_xgb = x_train.as_matrix()
y_train_xgb = y_train.as_matrix()

model = xgb.XGBClassifier()
model.fit(x_train_xgb, y_train_xgb)

y_probs = model.predict_proba(x_test.as_matrix())
print(y_probs)

final = [[iden,(y_probs[j][0]),(y_probs[j][1]),(y_probs[j][2])] for j,iden in enumerate(testUniqueIds)]
frame = pd.DataFrame(final,columns=('id','predict_0','predict_1','predict_2'))
frame.to_csv("submission.csv",index=False)

## Scikit

Going to build test harness of test, train, and validation splits using KFolding. Then run predicitions for each fold. Also want to keep track of true positive rate (tpr) and false positive rate (fpr) so I can evaluate receiving operating characteristic curves (ROC's) and the associated area under the curve (AUC) of each. I will also plot the confusion matrix for each model to visualize the predicitve results.

### Test Harness

Define the cross-validation and confusion matrix helper functions.

In [ ]:
# build a cross-validation function
@mem.cache
def run_cv(X, y, clf_class, **kwargs):
    kf = cross_validation.KFold(len(y), n_folds=10, shuffle=True)
    y_pred = y.copy()
    mean_acc = 0.0
    mean_prec = 0.0
    mean_recl = 0.0
    mean_f1 = 0.0
    for train_index, test_index in kf:
        clf = clf_class(**kwargs)
        X_train, X_test = X.ix[train_index], X.ix[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        y_pred[test_index] = clf.predict(X_test)
        mean_acc += metrics.accuracy_score(y[test_index], y_pred[test_index])
        mean_recl += metrics.recall_score(y[test_index], y_pred[test_index])
        mean_prec += metrics.precision_score(y[test_index], y_pred[test_index])
        mean_f1 += metrics.f1_score(y[test_index], y_pred[test_index])
    mean_acc /= len(kf)
    mean_recl /= len(kf)
    mean_prec /= len(kf)
    mean_f1 /= len(kf)
    return y_pred, mean_acc, mean_prec, mean_recl, mean_f1

def draw_confusion_matrices(confusion_matrices, class_names):
    labels = list(class_names)
    for cm in confusion_matrices:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        cax = ax.matshow(cm[1])
        fig.colorbar(cax)
        ax.set_xticklabels([''] + labels)
        ax.set_yticklabels([''] + labels)
        plt.title('%s' % cm[0])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        for i,j in ((x,y) for x in xrange(len(cm[1])) for y in xrange(len(cm[1][0]))):
            val = '{:.1%}'.format(cm[1][i][j]/cm[1].sum())
            ax.annotate(str(val), xy=(i,j), color='white', ha="center", va="center")
        plt.show()

### Models

Loading up a bunch of standard machine learning algorithms into test harness. This is a classification problem, so let's try the standard models...

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.ensemble import RandomForestClassifier as RF

ml_models = []
ml_models.append(["Support Vector", SVC, {'verbose':1}])
ml_models.append(["Logistic Regression", LR, {'multi_class':'multinomial',
                                              'solver':'lbfgs',
                                              'verbose':1}])
ml_models.append(["Naive Bayes", NB, {'verbose':1}])
ml_models.append(["Decision Trees", DT, {'verbose':1}])
ml_models.append(["Random Forest", RF, {'n_estimators':200,
                                        'min_samples_split':2,
                                        'n_jobs':-1,
                                        'verbose':1}])

### Measure Performance

Show me the accuracy (acc), precision (prec), recall (recl), confusion matrix (cm), and mean auc for each model. 

**TODO:**
* Gut check accuracy for equivalence:
    ```python
    metrics.jaccard_similarity_score(y_true, y_pred) == metrics.accuracy_score(y_true, y_pred)
    metrics.classification_report(y, y_pred)
    metrics.confusion_matrix(y, y_pred)
    ``` 

In [ ]:
# process each model
models = []
acc = []
prec = []
recl = []
f1 = []
cms = []
for m in ml_models:
    y_pred, mean_acc, mean_prec, mean_recl, mean_f1 = run_cv(X, y, m[1], **m[2])
    models.append(m[0])
    acc.append(mean_acc)
    prec.append(mean_prec)
    recl.append(mean_recl)
    f1.append(mean_f1)
    cms.append((m[0], metrics.confusion_matrix(y, y_pred)))

# plot confusion matricies
draw_confusion_matrices(cms, np.unique(y))

In [ ]:
models_df = pd.DataFrame.from_items([('model', models),
                              ('accuracy', acc),
                              ('precision', prec),
                              ('recall', recl),
                              ('f1', f1)
                              ])
models_df.sort(['accuracy', 'precision', 'recall', 'f1'], ascending=[False, False, False, False])

### Testing Results

Find which of the harness models has performed the best in the first pass.

In [ ]:
# best and worst models
max_idx, max_value = max(enumerate(acc), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best accuracy:', ml_models[max_idx][0], acc[max_idx])

max_idx, max_value = max(enumerate(prec), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best precision:', ml_models[max_idx][0], prec[max_idx])

max_idx, max_value = max(enumerate(recl), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best recall:', ml_models[max_idx][0], recl[max_idx])

Random forest is the obvious winner by all measures, so let's take a crack at tuning that model.

### Algorithm Tuning

#### Question: What are the important features in the model?

In [ ]:
# feature importance
kf = cross_validation.KFold(len(y), n_folds=10, shuffle=True, random_state=4)
y_pred = y.copy()

# keep values for all features
feat_impt = {}
for f in features:
    feat_impt[f] = []

# go through all features    
for train_index, test_index in kf:
    clf = RF(n_estimators=50, min_samples_split=2, n_jobs=-1)
    X_train, X_test = X.ix[train_index], X.ix[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    y_pred[test_index] = clf.predict(X_test)
    importances = clf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    for f in range(X.shape[1]):
        feat_impt[features[indices[f]]].append(importances[indices[f]])

for k in feat_impt:
    val = np.mean(feat_impt[k])
    std = np.std(feat_impt[k])
    feat_impt[k] = val

In [ ]:
# sort the features by importance
sorted_feat = sorted(feat_impt.items(), key=operator.itemgetter(1))
sorted_feat.reverse()

# bookkeeping
cumper = 0.
nbr = False

# make a copy of original dataframe to drop unneeded features
X_keep = X.copy()

# print out importances
print "item -- feature -- weight -- cumm_weight"
for i, k in enumerate(sorted_feat):
    cumper += k[1]
    
    # tell me when we hit 90% of weights
    if (nbr is False and cumper >= 0.95):
        print '**----  Reached 95%  ----**'
        nbr = True
    print i, k[0], '{:.1%}'.format(k[1]), '{:.0%}'.format(cumper)
    
    # drop unneeded features
    if nbr is True:
        X_keep = X_keep.drop(k[0], axis=1)

In [ ]:
features_keep = X_keep.columns

# checks
print "\n%d observations of %d features" % X_keep.shape
print "Unique labels:", np.unique(y)
print "Gut-check features: %i" % len(features_keep)

Re-run the ML test harness for RF only and see if the model improves (below is cut and paste of earlier code)

In [ ]:
ml_models = []
ml_models.append(["Random Forest", RF, {'n_estimators':72,
                                        'min_samples_split':9,
                                        'n_jobs':-1}])
acc = []
prec = []
recl = []
f1 = []
cms = []
for m in ml_models:
    y_pred, mean_acc, mean_prec, mean_recl, mean_f1 = run_cv(X_keep, y, m[1], **m[2])
    acc.append(mean_acc)
    prec.append(mean_prec)
    recl.append(mean_recl)
    f1.append(mean_f1)
    cms.append((m[0], metrics.confusion_matrix(y, y_pred)))

draw_confusion_matrices(cms, np.unique(y))
max_idx, max_value = max(enumerate(acc), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best accuracy:', ml_models[max_idx][0], acc[max_idx])
max_idx, max_value = max(enumerate(prec), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best precision:', ml_models[max_idx][0], prec[max_idx])
max_idx, max_value = max(enumerate(recl), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best recall:', ml_models[max_idx][0], recl[max_idx])

* RF predictive power got worse when features contributing < 10% weight were dropped.
* **RF predictive power got better when features contributing < 5% weight were dropped.**

#### Question: What model has best calibration and discrimination?

Calibration measures the difference between actual and predicted probability for individual groups. Discrimination measures the difference between model predictions and the baseline probability. Equations are taken from [Yang, Yates, and Smith (1991)](http://psychology.huji.ac.il/.upload/Ilan/YanivYatesSmith1991PB.pdf).

In [ ]:
def run_prob_cv(X, y, clf_class, **kwargs):
    kf = cross_validation.KFold(len(y), n_folds=10, shuffle=True)
    y_prob = np.zeros((len(y), 3))
    for train_index, test_index in kf:
        clf = clf_class(**kwargs)
        X_train, X_test = X.ix[train_index], X.ix[test_index]
        y_train = y[train_index]
        clf.fit(X_train,y_train)
        y_prob[test_index] = clf.predict_proba(X_test)
    return y_prob

def calibration(prob, outcome,n_bins=10):
    prob = np.array(prob)
    outcome = np.array(outcome)
    c = 0.0
    judgement_bins = np.arange(n_bins + 1) / n_bins
    bin_num = np.digitize(prob,judgement_bins)
    for j_bin in np.unique(bin_num):
        in_bin = bin_num == j_bin
        predicted_prob = np.mean(prob[in_bin])
        true_bin_prob = np.mean(outcome[in_bin])
        c += np.sum(in_bin) * ((predicted_prob - true_bin_prob) ** 2)
    return c / len(prob)

def discrimination(prob, outcome, n_bins=10):
    prob = np.array(prob)
    outcome = np.array(outcome)
    d = 0.0
    base_prob = np.mean(outcome)
    judgement_bins = np.arange(n_bins + 1) / n_bins
    bin_num = np.digitize(prob,judgement_bins)
    for j_bin in np.unique(bin_num):
        in_bin = bin_num == j_bin
        true_bin_prob = np.mean(outcome[in_bin])
        d += np.sum(in_bin) * ((true_bin_prob - base_prob) ** 2)
    return d / len(prob)

In [ ]:
print "Lower calibration and higher discrimination are preferred..."

ml_models = []
#ml_models.append(["XGBoost", XGBC, {'max_depth':3, 'n_estimators':300, 'learning_rate':0.05}])
#ml_models.append(["Support Vector Machine", SVC, {}])
ml_models.append(["Linear Logistic Regression", LR, {'multi_class':'multinomial',
                                                     'solver':'lbfgs'}])
ml_models.append(["Naive Bayes", NB, {}])
ml_models.append(["Decision Trees", DT, {}])
ml_models.append(["Random Forest", RF, {'n_estimators':200,
                                        'min_samples_split':2, 'n_jobs':-1}])

cal_err = []
discrim = []
for m in ml_models:
    print "\n"+m[0]
    pred_prob = run_prob_cv(X, y, m[1], **m[2])
    churn_prob, is_churn = pred_prob[:,1], y == 1
    cal_err.append(calibration(churn_prob, is_churn))
    discrim.append(discrimination(churn_prob,is_churn))
    print '{:20} {:.4f}'.format("Calibration Error", cal_err[-1])
    print '{:20} {:.4f}'.format("Discrimination", discrim[-1])
    
idx, value = min(enumerate(cal_err), key=operator.itemgetter(1))
print '{:20} {:20} {:.3%}'.format('\nBest calibration error:', ml_models[idx][0], cal_err[idx])
idx, value = max(enumerate(discrim), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best discrimination:', ml_models[idx][0], discrim[idx])

#### Question: Can we tune the model params to get better results?

In [ ]:
from operator import itemgetter

def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [ ]:
clf = RF(n_jobs=-1)
param_grid = {"n_estimators":[50, 75, 100],
              "max_depth": [None, 3],
              "max_features": [None, 'auto'],
              "min_samples_split": [4, 9],
              "min_samples_leaf": [1],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
gs = grid_search.GridSearchCV(clf, param_grid=param_grid)
start = time()
gs.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(gs.grid_scores_)))
report(grid_search.grid_scores_)

**Best scores using X (took 542.40 seconds for 32 candidate parameter settings):**

Model with rank: 1
Mean validation score: 0.956 (std: 0.007)
Parameters: {'bootstrap': True, 'min_samples_leaf': 1, 'n_estimators': 500, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 'auto', 'max_depth': None}

Model with rank: 2
Mean validation score: 0.956 (std: 0.009)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 1000, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 'auto', 'max_depth': None}

Model with rank: 3
Mean validation score: 0.955 (std: 0.006)
Parameters: {'bootstrap': True, 'min_samples_leaf': 1, 'n_estimators': 1000, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 'auto', 'max_depth': None}


**Best scores using X_keep (took 471.68 seconds for 48 candidate parameter settings):**

Model with rank: 1
Mean validation score: 0.957 (std: 0.007)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 1000, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 'auto', 'max_depth': None}

Model with rank: 2
Mean validation score: 0.956 (std: 0.006)
Parameters: {'bootstrap': True, 'min_samples_leaf': 1, 'n_estimators': 500, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 'auto', 'max_depth': None}

Model with rank: 3
Mean validation score: 0.956 (std: 0.007)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 50, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 'auto', 'max_depth': None}

In [ ]:
ml_models = []
ml_models.append(["Random Forest", RF, {'bootstrap': True,
                                        'min_samples_leaf': 1,
                                        'n_estimators': 500,
                                        'min_samples_split': 4,
                                        'criterion': 'entropy',
                                        'max_features': 'auto',
                                        'max_depth': None,
                                        'n_jobs': -1}])
acc = []
prec = []
recl = []
f1 = []
cms = []
for m in ml_models:
    y_pred, mean_acc, mean_prec, mean_recl, mean_f1 = run_cv(X, y, m[1], **m[2])
    acc.append(mean_acc)
    prec.append(mean_prec)
    recl.append(mean_recl)
    f1.append(mean_f1)
    cms.append((m[0], metrics.confusion_matrix(y, y_pred)))
draw_confusion_matrices(cms, np.unique(y))
max_idx, max_value = max(enumerate(acc), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best accuracy:', ml_models[max_idx][0], acc[max_idx])
max_idx, max_value = max(enumerate(prec), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best precision:', ml_models[max_idx][0], prec[max_idx])
max_idx, max_value = max(enumerate(recl), key=operator.itemgetter(1))
print '{:20} {:20} {:.2%}'.format('Best recall:', ml_models[max_idx][0], recl[max_idx])

**This model is hardly better than the initial guess, but it's technically simpler, so I'll run with this one.**

In [ ]:
# Use 10 estimators so predictions are all multiples of 0.1
m = (["Random Forest", RF, {'bootstrap': True,
                            'min_samples_leaf': 1,
                            'n_estimators': 10,
                            'min_samples_split': 1,
                            'criterion': 'entropy',
                            'max_features': 'auto',
                            'max_depth': None,
                            'n_jobs': -1}])
pred_prob = run_prob_cv(X, y, m[1], **m[2])
pred_churn = pred_prob[:,1]
is_churn = y == 1

# Number of times a predicted probability is assigned to an observation
counts = pd.value_counts(pred_churn)

# calculate true probabilities
true_prob = {}
for prob in counts.index:
    true_prob[prob] = np.mean(is_churn[pred_churn == prob])
    true_prob = pd.Series(true_prob)

# pandas-fu
counts = pd.concat([counts,true_prob], axis=1).reset_index()
counts.columns = ['pred_prob', 'count', 'true_prob']
counts

In [ ]:
from ggplot import *
baseline = np.mean(is_churn)
ggplot(counts, aes(x='pred_prob',y='true_prob',size='count')) + \
        geom_point(color='blue') + \
        stat_function(fun = lambda x: x, color='red') + \
        stat_function(fun = lambda x: baseline, color='green') + \
        xlim(-0.05,  1.05) + \
        ylim(-0.05,1.05) + \
        ggtitle("Random Forest") + \
        xlab("Predicted probability") + ylab("Relative frequency of outcome")

# Final Model

In [ ]:
# event, feature, resource, severity, volume, location

adc = pd.read_csv('adc.csv') #efrs
rf = pd.read_csv('randomforest.csv') #esv
loc = pd.read_csv('location.csv') # l
gbc = pd.read_csv('gbc.csv') # e
nn = pd.read_csv('neural_network.csv') # rs
xgbc = pd.read_csv('xgbc.csv') # efrsvl

df_final = rf.copy()
df_final.loc[:, 'predict_0':'predict_2'] = (
    5.0*rf.loc[:, 'predict_0':'predict_2'] + 
    3.0*loc.loc[:, 'predict_0':'predict_2'] +
    0.1*adc.loc[:, 'predict_0':'predict_2'] +
    0.2*nn.loc[:, 'predict_0':'predict_2'] +
    6.0*gbc.loc[:, 'predict_0':'predict_2'] +
    10.0*xgbc.loc[:, 'predict_0':'predict_2']
    )/24.3

df_final.to_csv('blend.csv', index=False)

In [ ]:
# save to hdf5 file
hdf = pd.HDFStore(hdf_file)
hdf.put('df_final', df_final)
hdf.close()
call(["lrztar", "-zf", hdf_file])

# Ad Hoc Junk

In [ ]:
# build random forest
clf = RandomForestClassifier(n_estimators=50, oob_score=True)  
n_trials = 100
test_size_percent = 0.1

# build labels
sig_field = 'sms_count'
signals = df[[c for c in df.columns if c != sig_field]]
labels = df[sig_field]

# test train split
plot_data = []
for trial in range(n_trials):
    train_signals, test_signals, train_labels, test_labels = train_test_split(
                                                            signals, labels, test_size=test_size_percent)
    clf.fit(train_signals, train_labels)
    predictions = clf.predict_proba(test_signals)[:,1]
    precision, recall, thresholds = precision_recall_curve(test_labels, predictions)  
    thresholds = np.append(thresholds, 1)
    queue_rate = []  
    for threshold in thresholds:  
        queue_rate.append((predictions >= threshold).mean())
        
    # save plot data
    plot_data.append({
            'thresholds': thresholds
        ,   'precision': precision
        ,   'recall': recall
        ,   'queue_rate': queue_rate
    })
    
# plot model
uniform_thresholds = np.linspace(0, 1, 101)
uniform_precision_plots = []  
uniform_recall_plots= []  
uniform_queue_rate_plots= []
for p in plot_data:  
    uniform_precision = []
    uniform_recall = []
    uniform_queue_rate = []
    for ut in uniform_thresholds:
        index = bisect.bisect_left(p['thresholds'], ut)
        uniform_precision.append(p['precision'][index])
        uniform_recall.append(p['recall'][index])
        uniform_queue_rate.append(p['queue_rate'][index])

    uniform_precision_plots.append(uniform_precision)
    uniform_recall_plots.append(uniform_recall)
    uniform_queue_rate_plots.append(uniform_queue_rate)

quantiles = [0.1, 0.5, 0.9]  
lower_precision, median_precision, upper_precision = mstats.mquantiles(uniform_precision_plots, quantiles, axis=0)  
lower_recall, median_recall, upper_recall = mstats.mquantiles(uniform_recall_plots, quantiles, axis=0)  
lower_queue_rate, median_queue_rate, upper_queue_rate = mstats.mquantiles(uniform_queue_rate_plots, quantiles, axis=0)

plt.plot(uniform_thresholds, median_precision)  
plt.plot(uniform_thresholds, median_recall)  
plt.plot(uniform_thresholds, median_queue_rate)

plt.fill_between(uniform_thresholds, upper_precision, lower_precision, alpha=0.5, linewidth=0, color=sns.color_palette()[0])  
plt.fill_between(uniform_thresholds, upper_recall, lower_recall, alpha=0.5, linewidth=0, color=sns.color_palette()[1])  
plt.fill_between(uniform_thresholds, upper_queue_rate, lower_queue_rate, alpha=0.5, linewidth=0, color=sns.color_palette()[2])

leg = plt.legend(('precision', 'recall', 'queue_rate'), frameon=True)  
leg.get_frame().set_edgecolor('k')  
plt.xlabel('threshold')  
plt.ylabel('%')  

uniform_payout_plots = []
n = 10000  
success_payoff = 100  
case_cost = 20

for p in plot_data:  
    uniform_payout = []
    for ut in uniform_thresholds:
        index = bisect.bisect_left(p['thresholds'], ut)
        precision = p['precision'][index]
        queue_rate = p['queue_rate'][index]
        payout = n*queue_rate*(precision*100 - case_cost)
        uniform_payout.append(payout)
    uniform_payout_plots.append(uniform_payout)

quantiles = [0.1, 0.5, 0.9]
lower_payout, median_payout, upper_payout = mstats.mquantiles(uniform_payout_plots, quantiles, axis=0)

plt.plot(uniform_thresholds, median_payout, color=sns.color_palette()[4])  
plt.fill_between(uniform_thresholds, upper_payout, lower_payout, alpha=0.5, linewidth=0, color=sns.color_palette()[4])

max_ap = uniform_thresholds[np.argmax(median_payout)]  
plt.vlines([max_ap], -100000, 150000, linestyles='--')  
plt.ylim(-100000, 150000)

leg = plt.legend(('payout ($)', 'median argmax = {:.2f}'.format(max_ap)), frameon=True)  
leg.get_frame().set_edgecolor('k')  
plt.xlabel('threshold')  
plt.ylabel('$')  
plt.title("Payout as a Function of Threshold")

print '{0:.0f}'.format(np.max(median_payout))

# Scratch Space